# Sentiment analysis with hotel reviews

## Processing the data

#### Load the data

In [2]:
# Load the hotel reviews from CSV
import pandas as pd
import time
# importing time so the start and end time can be used to calculate file loading time
print("Loading data file now, this could take a while depending on file size")
start = time.time()
# df is 'DataFrame' - make sure you downloaded the file to the data folder
df = pd.read_csv('Hotel_Reviews.csv')
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds")

Loading data file now, this could take a while depending on file size
Loading took 1.48 seconds


#### Dataframe operations

In [10]:
# 1. Print out the shape of the data frame you have just loaded (the shape is the number of rows and columns)
df.shape
# print(df.columns)
# the followings are the columns in the dataframe
# Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
#        'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
#        'Negative_Review', 'Review_Total_Negative_Word_Counts',
#        'Total_Number_of_Reviews', 'Positive_Review',
#        'Review_Total_Positive_Word_Counts',
#        'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
#        'days_since_review', 'lat', 'lng'],
#       dtype='object')

# 2 Calculate the frequency count for reviewer nationalities
# 2.1 How many distinct values are there for the column Reviewer_Nationality and what are they?
nationality_freq = df["Reviewer_Nationality"].value_counts()
# print(nationality_freq)
# 2.2 What reviewer nationality are the top 10 in the dataset (print country and number of reviews)?
# print(nationality_freq[0:10].to_string())

# 3 What was the most frequently reviewed hotel for each of the top 10 most reviewer nationalities
for nat in nationality_freq[:10].index:
    # First, extract all the rows that match the criteria into a new dataframe
    nat_df = df[df["Reviewer_Nationality"] == nat]
    # Now get the hotel freq
    freq = nat_df["Hotel_Name"].value_counts()
    # print("[" + str(nat).strip() + "] was [" + str(freq.index[0]) + "] with [" + str(freq[0]) + "] reviews.")

# 4 How many reviews are there per hotel (frequency count of hotel) in the dataset?
nationality_freq = df["Hotel_Name"].value_counts()
# print(nationality_freq)
# print(nationality_freq[0:10].to_string())

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# 5 calculate an average score. Add a new column to your dataframe with the column header Calc_Average_Score that contains that calculated average.
df['Calc_Average_Score'] = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)
# print(df['Calc_Average_Score'][0:10].to_string())

In [11]:
# 6 Do any hotels have the same Average_Score and Calc_Average_Score?
# define a function that takes a row and performs some calculation with it
def get_difference_review_avg(row):
    return row["Average_Score"] - row["Calc_Average_Score"]
# Add a new column with the difference between the two average scores
df["Average_Score_Difference"] = df.apply(get_difference_review_avg, axis = 1)
# print hotels that have the same average score
same_avg = df[df["Average_Score_Difference"] == 0]
print(same_avg[["Hotel_Name", "Average_Score", "Calc_Average_Score"]][0:10].to_string(index=False))

                      Hotel_Name  Average_Score  Calc_Average_Score
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7
The Park Grand London Paddington            7.7                 7.7


In [34]:
# 7 Calculate and print out how many rows have column Negative_Review values of "No Negative"
no_negative_reviews = df[df['Negative_Review'] == 'No Negative']
# print(no_negative_reviews[['Hotel_Name', 'Negative_Review']][0:20].to_string())
print('No Negative counts: ' + str(len(no_negative_reviews.index)))

# 8 Calculate and print out how many rows have column Positive_Review values of "No Positive"
no_positive_reviews = df[df['Positive_Review'] == 'No Positive']
# print(no_positive_reviews[['Hotel_Name', 'Positive_Review']][0:20].to_string())
print('No Positive counts: ' + str(len(no_positive_reviews.index)))

# 9 Calculate and print out how many rows have column Negative_Review values of "No Negative" and Positive_Review values of "No Positive"
no_reviews = df[(df['Negative_Review'] == 'No Negative') & (df['Positive_Review'] == 'No Positive')]
print('No Positive counts: ' + str(len(no_reviews.index)))
print(no_reviews[['Hotel_Name', 'Negative_Review', 'Positive_Review']][0:20].to_string())

No Negative counts: 127890
No Positive counts: 35946
No Positive counts: 127
                                       Hotel_Name Negative_Review Positive_Review
1403                      Apex Temple Court Hotel     No Negative     No Positive
7272                Park Plaza County Hall London     No Negative     No Positive
7892                    Hotel Trianon Rive Gauche     No Negative     No Positive
21443                 Novotel London Tower Bridge     No Negative     No Positive
22770                   W London Leicester Square     No Negative     No Positive
27079                               The Grosvenor     No Negative     No Positive
27135                               The Grosvenor     No Negative     No Positive
33812                          11 Cadogan Gardens     No Negative     No Positive
40033                                  Arbor City     No Negative     No Positive
52650                    Hilton London Paddington     No Negative     No Positive
55638                

## Building a sentiment analysis model

#### Clean the data

In [ ]:
# 1 drop the columns don't need
df = df.drop(['lat', 'lng', 'Additional_Number_of_Scoring','Review_Total_Negative_Word_Counts', 'Review_Total_Positive_Word_Counts', 'Review_Date', 'days_since_review', 'Total_Number_of_Reviews_Reviewer_Has_Given'], axis=1)

In [41]:
# 2 Replace Hotel_Address values with just the city and country
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria"
df["Hotel_Address"] = df.apply(replace_address, axis = 1)
print(df["Hotel_Address"].value_counts())
display(df.groupby("Hotel_Address").agg({"Hotel_Name": "nunique"}))

Hotel_Address
London, United Kingdom    262301
Barcelona, Spain           60149
Paris, France              59928
Amsterdam, Netherlands     57214
Vienna, Austria            38939
Milan, Italy               37207
Name: count, dtype: int64


,Hotel_Name
Hotel_Address,
"Amsterdam, Netherlands",105
"Barcelona, Spain",211
"London, United Kingdom",400
"Milan, Italy",162
"Paris, France",458
"Vienna, Austria",158


In [44]:
# 3 Replace Total_Number_of_Reviews, Average_Score with actual data in the dataset
df.Total_Number_of_Reviews = df.groupby('Hotel_Name').transform('count')
df.Average_Score = round(df.groupby('Hotel_Name').Reviewer_Score.transform('mean'), 1)

#### Filtering tags

In [45]:
# Remove opening and closing brackets
df.Tags = df.Tags.str.strip("[']")
# remove all quotes too
df.Tags = df.Tags.str.replace(" ', '", ",", regex = False)

In [46]:
# Process the Tags into new columns
df["Leisure_trip"] = df.Tags.apply(lambda tag: 1 if "Leisure trip" in tag else 0)
df["Couple"] = df.Tags.apply(lambda tag: 1 if "Couple" in tag else 0)
df["Solo_traveler"] = df.Tags.apply(lambda tag: 1 if "Solo traveler" in tag else 0)
df["Business_trip"] = df.Tags.apply(lambda tag: 1 if "Business trip" in tag else 0)
df["Group"] = df.Tags.apply(lambda tag: 1 if "Group" in tag or "Travelers with friends" in tag else 0)
df["Family_with_young_children"] = df.Tags.apply(lambda tag: 1 if "Family with young children" in tag else 0)
df["Family_with_older_children"] = df.Tags.apply(lambda tag: 1 if "Family with older children" in tag else 0)
df["With_a_pet"] = df.Tags.apply(lambda tag: 1 if "With a pet" in tag else 0)

In [51]:
df["Leisure_trip"].value_counts()
# count the number of rows for Leisure_trip == 1
print("Leisure trip counts: " + str(len(df[df["Leisure_trip"] == 1].index)))

Leisure trip counts: 417778


#### Save your file

In [56]:
# Saving new data file with calculated columns
df.to_csv(r'Hotel_Reviews_Filtered.csv', index = False)

## Sentiment Analysis Operations

#### load and save the filtered data

In [61]:
import time
import pandas as pd
import nltk as nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Load the filtered hotel reviews from CSV
new_df = pd.read_csv('Hotel_Reviews_Filtered.csv')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lvgy\AppData\Roaming\nltk_data...


#### Removing stop words

In [62]:

# Remove stop words - can be slow for a lot of text!
# Ryan Han (ryanxjhan on Kaggle) has a great post measuring performance of different stop words removal approaches
# https://www.kaggle.com/ryanxjhan/fast-stop-words-removal # using the approach that Ryan recommends
start = time.time()
cache = set(stopwords.words("english"))
def remove_stopwords(review):
    text = " ".join([word for word in review.split() if word not in cache])
    return text

# Remove the stop words from both columns
new_df.Negative_Review = new_df.Negative_Review.apply(remove_stopwords)
new_df.Positive_Review = new_df.Positive_Review.apply(remove_stopwords)

#### Performing sentiment analysis

In [63]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create the vader sentiment analyser (there are others in NLTK you can try too)
vader_sentiment = SentimentIntensityAnalyzer()
# Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

# There are 3 possibilities of input for a review:
# It could be "No Negative", in which case, return 0
# It could be "No Positive", in which case, return 0
# It could be a review, in which case calculate the sentiment
def calc_sentiment(review):
    if review == "No Negative" or review == "No Positive":
        return 0
    return vader_sentiment.polarity_scores(review)["compound"]

In [65]:
# Add a negative sentiment and positive sentiment column
print("Calculating sentiment columns for both positive and negative reviews")
start = time.time()
new_df["Negative_Sentiment"] = new_df.Negative_Review.apply(calc_sentiment)
new_df["Positive_Sentiment"] = new_df.Positive_Review.apply(calc_sentiment)
end = time.time()
print("Calculating sentiment took " + str(round(end - start, 2)) + " seconds")

Calculating sentiment columns for both positive and negative reviews
Calculating sentiment took 97.01 seconds


In [67]:
# to print of the results and see if the sentiment matches the review
new_df = new_df.sort_values(by=["Negative_Sentiment"], ascending=True)
print(new_df[["Negative_Review", "Negative_Sentiment"]])
new_df = new_df.sort_values(by=["Positive_Sentiment"], ascending=True)
print(new_df[["Positive_Review", "Positive_Sentiment"]])

                                          Negative_Review  Negative_Sentiment
186584  So bad experience memories I hotel The first n...             -0.9920
129503  First charged twice room booked booking second...             -0.9896
307286  The staff Had bad experience even booking Janu...             -0.9889
201953  Everything DO NOT STAY AT THIS HOTEL I never i...             -0.9886
452092  No WLAN room Incredibly rude restaurant staff ...             -0.9884
...                                                   ...                 ...
138365  Wifi terribly slow I speed test network upload...              0.9938
79215   I find anything hotel first I walked past hote...              0.9938
278506  The property great location There bakery next ...              0.9945
339189  Guys I like hotel I wish return next year Howe...              0.9948
480509  I travel lot far visited countless number hote...              0.9957

[515738 rows x 2 columns]
                                     

In [68]:
# save
# Reorder the columns (This is cosmetic, but to make it easier to explore the data later)
new_df = new_df.reindex(["Hotel_Name", "Hotel_Address", "Total_Number_of_Reviews", "Average_Score", "Reviewer_Score", "Negative_Sentiment", "Positive_Sentiment", "Reviewer_Nationality", "Leisure_trip", "Couple", "Solo_traveler", "Business_trip", "Group", "Family_with_young_children", "Family_with_older_children", "With_a_pet", "Negative_Review", "Positive_Review"], axis=1)

print("Saving results to Hotel_Reviews_NLP.csv")
new_df.to_csv(r"Hotel_Reviews_NLP.csv", index = False)

Saving results to Hotel_Reviews_NLP.csv
